In [1]:
#!/usr/bin/env python3
import os
import csv
import sys
import glob
from pathlib import Path

def parse_zsim_file(zsim_file):
    """Parse a single zsim file and return metrics"""
    
    # Initialize all metrics
    instructions = 0
    cycles = 0
    cycles_list = []
    
    l3_misses = l2_misses = l1_misses = l1i_misses = l1d_misses = 0
    l3_hits = l2_hits = l1_hits = l1i_hits = l1d_hits = 0
    
    l1d = l1i = l2d = l3d = False
    
    # Energy constants
    ENERGY_L1I_HIT, ENERGY_L1I_MISS = 15.0, 33.0
    ENERGY_L1D_HIT, ENERGY_L1D_MISS = 15.0, 33.0
    ENERGY_L2_HIT, ENERGY_L2_MISS = 46.0, 93.0
    ENERGY_L3_HIT, ENERGY_L3_MISS = 945.0, 1904.0
    ENERGY_MAIN_MEMORY = 2.0
    ENERGY_PER_CYCLE = 0.1
    CLOCK_FREQUENCY_GHz = 2.4
    
    # Parse the file
    with open(zsim_file, "r") as ins:
        for line in ins:
            try:
                # CPU Metrics
                if "instrs: " in line:
                    instructions += int(line.split()[1])
                if "Simulated unhalted cycles" in line:
                    cycles_list.append(int(line.split()[1]))
                
                # Cache level detection
                if "l1i:" in line: l1i, l1d, l2d, l3d = True, False, False, False
                elif "l1d:" in line: l1d, l1i, l2d, l3d = True, False, False, False
                elif "l2: # Cache stats" in line: l2d, l1d, l1i, l3d = True, False, False, False
                elif "l3: # Cache stats" in line: l3d, l1d, l1i, l2d = True, False, False, False
                elif "sched: # Scheduler stats" in line: l1d = l1i = l2d = l3d = False
                
                # Parse cache stats
                if l1i and "# GETS hits" in line:
                    l1i_hits += int(line.split()[1])
                elif l1i and "# GETS misses" in line:
                    l1i_misses += int(line.split()[1])
                elif l1d and ("# GETS hits" in line or "# GETX hits" in line):
                    l1d_hits += int(line.split()[1])
                    l1_hits += int(line.split()[1])
                elif l1d and ("# GETS misses" in line or "# GETX I->M misses" in line):
                    l1d_misses += int(line.split()[1])
                    l1_misses += int(line.split()[1])
                elif l2d and ("hGETS:" in line or "hGETX:" in line):
                    l2_hits += int(line.split()[1])
                elif l2d and "# GETS misses" in line:
                    l2_misses += int(line.split()[1])
                elif l3d and ("hGETS:" in line or "hGETX:" in line):
                    l3_hits += int(line.split()[1])
                elif l3d and ("# GETS misses" in line or "# GETX I->M misses" in line):
                    l3_misses += int(line.split()[1])
            except:
                continue
    
    # Calculate all metrics
    cycles = max(cycles_list) if cycles_list else 0
    ipc = float(instructions) / float(cycles) if cycles > 0 else 0.0
    
    # Miss rates
    l1i_miss_rate = (l1i_misses / float(l1i_misses + l1i_hits)) * 100.0 if (l1i_misses + l1i_hits) > 0 else 0.0
    l1d_miss_rate = (l1d_misses / float(l1d_misses + l1d_hits)) * 100.0 if (l1d_misses + l1d_hits) > 0 else 0.0
    l1_miss_rate = (l1_misses / float(l1_misses + l1_hits)) * 100.0 if (l1_misses + l1_hits) > 0 else 0.0
    l2_miss_rate = (l2_misses / float(l2_misses + l2_hits)) * 100.0 if (l2_misses + l2_hits) > 0 else 0.0
    l3_miss_rate = (l3_misses / float(l3_misses + l3_hits)) * 100.0 if (l3_misses + l3_hits) > 0 else 0.0
    
    # MPKI
    l1_mpki = l1_misses / float(instructions / 1000.0) if instructions > 0 else 0.0
    l2_mpki = l2_misses / float(instructions / 1000.0) if instructions > 0 else 0.0
    l3_mpki = l3_misses / float(instructions / 1000.0) if instructions > 0 else 0.0
    
    # LFMR
    lfmr = float(l3_misses) / float(l1_misses) if l1_misses > 0 else 0.0
    
    # Energy calculations
    energy_l1i = (l1i_hits * ENERGY_L1I_HIT) + (l1i_misses * ENERGY_L1I_MISS)
    energy_l1d = (l1d_hits * ENERGY_L1D_HIT) + (l1d_misses * ENERGY_L1D_MISS)
    energy_l2 = (l2_hits * ENERGY_L2_HIT) + (l2_misses * ENERGY_L2_MISS)
    energy_l3 = (l3_hits * ENERGY_L3_HIT) + (l3_misses * ENERGY_L3_MISS)
    energy_memory = l3_misses * ENERGY_MAIN_MEMORY
    energy_cpu_core = cycles * ENERGY_PER_CYCLE
    
    total_cache_energy = energy_l1i + energy_l1d + energy_l2 + energy_l3
    total_system_energy = total_cache_energy + energy_memory + energy_cpu_core
    
    # Power calculations
    execution_time_seconds = cycles / (CLOCK_FREQUENCY_GHz * 1e9)
    average_power_watts = (total_system_energy * 1e-12) / execution_time_seconds if execution_time_seconds > 0 else 0.0
    perf_power_ratio = ipc / average_power_watts if average_power_watts > 0 else 0.0
    
    return {
        'instructions': instructions, 'cycles': cycles, 'ipc': round(ipc, 4),
        'execution_time': round(execution_time_seconds, 6),
        'l1i_hits': l1i_hits, 'l1i_misses': l1i_misses, 'l1i_miss_rate': round(l1i_miss_rate, 2),
        'l1d_hits': l1d_hits, 'l1d_misses': l1d_misses, 'l1d_miss_rate': round(l1d_miss_rate, 2),
        'l1_hits': l1_hits, 'l1_misses': l1_misses, 'l1_miss_rate': round(l1_miss_rate, 2),
        'l2_hits': l2_hits, 'l2_misses': l2_misses, 'l2_miss_rate': round(l2_miss_rate, 2),
        'l3_hits': l3_hits, 'l3_misses': l3_misses, 'l3_miss_rate': round(l3_miss_rate, 2),
        'l1_mpki': round(l1_mpki, 2), 'l2_mpki': round(l2_mpki, 2), 'l3_mpki': round(l3_mpki, 2),
        'lfmr': round(lfmr, 4),
        'energy_l1i': round(energy_l1i, 2), 'energy_l1d': round(energy_l1d, 2),
        'energy_l2': round(energy_l2, 2), 'energy_l3': round(energy_l3, 2),
        'energy_memory': round(energy_memory, 2), 'energy_cpu_core': round(energy_cpu_core, 2),
        'total_cache_energy': round(total_cache_energy, 2), 'total_system_energy': round(total_system_energy, 2),
        'average_power_mw': round(average_power_watts * 1000, 4),
        'perf_power_ratio': round(perf_power_ratio * 1000, 4)
    }

def main():
    print("🚀 Lazy ZSim Batch Processor 🚀")
    print("================================")
    
    # Find all .out files in current directory
    out_files = glob.glob("*.coffee")
    
    if not out_files:
        print("❌ No .txt files found in current directory!")
        print("Make sure you're in the directory with your zsim output files.")
        return
    
    print(f"📁 Found {len(out_files)} .coffee files:")
    for f in out_files:
        print(f"   • {f}")
    
    # Ask for output CSV name
    csv_filename = input("\n📝 Enter output CSV filename (default: all_results.csv): ").strip()
    if not csv_filename:
        csv_filename = "all_results.csv"
    
    if not csv_filename.endswith('.csv'):
        csv_filename += '.csv'
    
    # CSV headers
    headers = [
        'Workload', 'Instructions', 'Cycles', 'IPC', 'Execution_Time_s',
        'L1I_Hits', 'L1I_Misses', 'L1I_Miss_Rate_%',
        'L1D_Hits', 'L1D_Misses', 'L1D_Miss_Rate_%',
        'L1_Hits', 'L1_Misses', 'L1_Miss_Rate_%',
        'L2_Hits', 'L2_Misses', 'L2_Miss_Rate_%',
        'L3_Hits', 'L3_Misses', 'L3_Miss_Rate_%',
        'L1_MPKI', 'L2_MPKI', 'L3_MPKI', 'LFMR',
        'L1I_Energy_pJ', 'L1D_Energy_pJ', 'L2_Energy_pJ', 'L3_Energy_pJ',
        'Memory_Energy_pJ', 'CPU_Core_Energy_pJ', 
        'Total_Cache_Energy_pJ', 'Total_System_Energy_pJ',
        'Average_Power_mW', 'Perf_Power_Ratio_IPC_per_mW'
    ]
    
    # Process all files
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        
        for i, out_file in enumerate(out_files, 1):
            print(f"\n⚡ Processing {i}/{len(out_files)}: {out_file}")
            
            try:
                # Generate workload name from filename
                workload_name = Path(out_file).stem  # removes .out extension
                
                # Parse the file
                metrics = parse_zsim_file(out_file)
                
                # Prepare data row
                row = [workload_name] + [
                    metrics['instructions'], metrics['cycles'], metrics['ipc'], metrics['execution_time'],
                    metrics['l1i_hits'], metrics['l1i_misses'], metrics['l1i_miss_rate'],
                    metrics['l1d_hits'], metrics['l1d_misses'], metrics['l1d_miss_rate'],
                    metrics['l1_hits'], metrics['l1_misses'], metrics['l1_miss_rate'],
                    metrics['l2_hits'], metrics['l2_misses'], metrics['l2_miss_rate'],
                    metrics['l3_hits'], metrics['l3_misses'], metrics['l3_miss_rate'],
                    metrics['l1_mpki'], metrics['l2_mpki'], metrics['l3_mpki'], metrics['lfmr'],
                    metrics['energy_l1i'], metrics['energy_l1d'], metrics['energy_l2'], metrics['energy_l3'],
                    metrics['energy_memory'], metrics['energy_cpu_core'],
                    metrics['total_cache_energy'], metrics['total_system_energy'],
                    metrics['average_power_mw'], metrics['perf_power_ratio']
                ]
                
                writer.writerow(row)
                
                # Show quick stats
                print(f"   ✅ IPC: {metrics['ipc']}, Power: {metrics['average_power_mw']} mW, Perf/Power: {metrics['perf_power_ratio']} IPC/mW")
                
            except Exception as e:
                print(f"   ❌ Error processing {out_file}: {e}")
                continue
    
    print(f"\n🎉 Done! All results saved to: {csv_filename}")
    print(f"📊 Processed {len(out_files)} files successfully!")
    print(f"\n💡 You can now open {csv_filename} in Excel/LibreOffice or analyze with Python/R")

if __name__ == "__main__":
    main()

🚀 Lazy ZSim Batch Processor 🚀
📁 Found 5 .coffee files:
   • lsvm_client_host_final.coffee
   • lsvm_client_pim_compute_final.coffee
   • lsvm_client_pim_data_final.coffee
   • lsvm_client_pim_network_final.coffee
   • lsvm_pim_full.coffee

⚡ Processing 1/5: lsvm_client_host_final.coffee
   ✅ IPC: 1.8211, Power: 10.003 mW, Perf/Power: 182059.0991 IPC/mW

⚡ Processing 2/5: lsvm_client_pim_compute_final.coffee
   ✅ IPC: 1.5543, Power: 3.673 mW, Perf/Power: 423166.5312 IPC/mW

⚡ Processing 3/5: lsvm_client_pim_data_final.coffee
   ✅ IPC: 1.5565, Power: 3.7651 mW, Perf/Power: 413400.7645 IPC/mW

⚡ Processing 4/5: lsvm_client_pim_network_final.coffee
   ✅ IPC: 1.5565, Power: 3.7651 mW, Perf/Power: 413397.7628 IPC/mW

⚡ Processing 5/5: lsvm_pim_full.coffee
   ✅ IPC: 1.5554, Power: 3.765 mW, Perf/Power: 413129.0155 IPC/mW

🎉 Done! All results saved to: all_results.csv
📊 Processed 5 files successfully!

💡 You can now open all_results.csv in Excel/LibreOffice or analyze with Python/R
